**Imports**

In [22]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

**Pre-Processing**

In [24]:
data = pd.read_csv('../data/WildfireData.csv')
data.head()

,Unnamed: 0,Unnamed: 0.1,fire_name,fire_size,fire_size_class,stat_cause_descr,latitude,longitude,state,disc_clean_date,...,Wind_cont,Hum_pre_30,Hum_pre_15,Hum_pre_7,Hum_cont,Prec_pre_30,Prec_pre_15,Prec_pre_7,Prec_cont,remoteness
0,0,0,NaN,10.0,C,Missing/Undefined,18.105072,-66.753044,PR,2/11/2007,...,3.250413,78.216590,76.793750,76.381579,78.724370,0.0,0.0,0.0,0.0,0.017923
1,1,1,NaN,3.0,B,Arson,35.038330,-87.610000,TN,12/11/2006,...,2.122320,70.840000,65.858911,55.505882,81.682678,59.8,8.4,0.0,86.8,0.184355
2,2,2,NaN,60.0,C,Arson,34.947800,-88.722500,MS,2/29/2004,...,3.369050,75.531629,75.868613,76.812834,65.063800,168.8,42.2,18.1,124.5,0.194544
3,3,3,WNA 1,1.0,B,Debris Burning,39.641400,-119.308300,NV,6/6/2005,...,0.000000,44.778429,37.140811,35.353846,0.000000,10.4,7.2,0.0,0.0,0.487447
4,4,4,NaN,2.0,B,Miscellaneous,30.700600,-90.591400,LA,9/22/1999,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,0.214633


In [25]:
data = data[['fire_size', 'remoteness', 'putout_time', 'stat_cause_descr']]
data.head()

,fire_size,remoteness,putout_time,stat_cause_descr
0,10.0,0.017923,NaN,Missing/Undefined
1,3.0,0.184355,NaN,Arson
2,60.0,0.194544,NaN,Arson
3,1.0,0.487447,0 days 00:00:00.000000000,Debris Burning
4,2.0,0.214633,NaN,Miscellaneous


In [26]:
data = data.dropna()
data['putout_time'] = data['putout_time'].str.split(' ').str[0]
data.head()

,fire_size,remoteness,putout_time,stat_cause_descr
3,1.0,0.487447,0,Debris Burning
9,1.0,0.291683,0,Miscellaneous
11,37.0,0.191923,0,Arson
14,3.0,0.123518,0,Missing/Undefined
22,2.0,0.054988,0,Miscellaneous


In [27]:
data.stat_cause_descr.unique()

array(['Debris Burning', 'Miscellaneous', 'Arson', 'Missing/Undefined',
       'Lightning', 'Equipment Use', 'Campfire', 'Structure', 'Children',
       'Smoking', 'Powerline', 'Railroad', 'Fireworks'], dtype=object)

In [28]:
data['stat_cause_descr'] = data['stat_cause_descr'].map({
    'Debris Burning': 0,
    'Arson': 1,
    'Lightning': 2,
    'Equipment Use': 3,
    'Campfire': 4,
    'Structure': 5,
    'Children': 5,
    'Smoking': 5,
    'Powerline': 5,
    'Railroad': 5,
    'Fireworks': 5,
    'Missing/Undefined': 5,
    'Miscellaneous': 5
})
data.stat_cause_descr.unique()
data.head()

,fire_size,remoteness,putout_time,stat_cause_descr
3,1.0,0.487447,0,0
9,1.0,0.291683,0,5
11,37.0,0.191923,0,1
14,3.0,0.123518,0,5
22,2.0,0.054988,0,5


In [29]:
from sklearn.model_selection import train_test_split
X = data.drop('stat_cause_descr', axis=1).astype('float')
y = data.stat_cause_descr.astype('float')
print(X.head())
print(y.head())
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

    fire_size  remoteness  putout_time
3         1.0    0.487447          0.0
9         1.0    0.291683          0.0
11       37.0    0.191923          0.0
14        3.0    0.123518          0.0
22        2.0    0.054988          0.0
3     0.0
9     5.0
11    1.0
14    5.0
22    5.0
Name: stat_cause_descr, dtype: float64


In [30]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [31]:
 y_test[0]

array([0., 0., 0., 0., 0., 1.], dtype=float32)

**Model**

In [32]:
model = Sequential()
model.add(Dense(32, input_dim=(3), activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(6, activation='softmax'))

In [33]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [34]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=64, verbose=1)

Epoch 1/100
387/387 [==============================] - 2s 2ms/step - loss: 117.5789 - accuracy: 0.2675 - val_loss: 3.6883 - val_accuracy: 0.3759
Epoch 2/100
387/387 [==============================] - 0s 1ms/step - loss: 6.7753 - accuracy: 0.3594 - val_loss: 6.7817 - val_accuracy: 0.3817
Epoch 3/100
387/387 [==============================] - 0s 1ms/step - loss: 7.0299 - accuracy: 0.3665 - val_loss: 3.6930 - val_accuracy: 0.3242
Epoch 4/100
387/387 [==============================] - 0s 1ms/step - loss: 7.0021 - accuracy: 0.3700 - val_loss: 1.9069 - val_accuracy: 0.3719
Epoch 5/100
387/387 [==============================] - 0s 1ms/step - loss: 3.8940 - accuracy: 0.3617 - val_loss: 5.1894 - val_accuracy: 0.3857
Epoch 6/100
387/387 [==============================] - 0s 1ms/step - loss: 4.0716 - accuracy: 0.3702 - val_loss: 6.1473 - val_accuracy: 0.3861
Epoch 7/100
387/387 [==============================] - 0s 1ms/step - loss: 6.4513 - accuracy: 0.3831 - val_loss: 3.4172 - val_accuracy: 0.39

In [35]:
# test model
test_data = {
    'fire_size': [37.0],
    'remoteness': [0.191923],
    'putout_time': [0]	
}
test_data = pd.DataFrame(test_data)
test_data

,fire_size,remoteness,putout_time
0,37.0,0.191923,0


In [36]:
result = model.predict(test_data)
result

array([[0.26619574, 0.42578152, 0.03273044, 0.03664317, 0.02949393,
        0.20915516]], dtype=float32)

In [37]:
# find class
final = 0
for i in range(6):
    if (result[0][i] > final):
        final = result[0][i]
    # print(final)
final = np.where(result[0] == final)
print(final[0][0])
    

1


In [38]:
# save model
model.save("../Weights/cause.h5")

In [39]:
# add cause to predicted fires dataset
predicted_data = pd.read_csv('../data/PredictedData.csv')

In [43]:
predicted_data = predicted_data.dropna()
predicted_data['putout_time'] = predicted_data['putout_time'].str.split(' ').str[0]
test_data = predicted_data[['fire_size', 'remoteness', 'putout_time']].astype('float')
results = model.predict(test_data)